<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Spectral_Unmixing_Italy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=p-vAOW8yJYMRJHHXCN457qDo1at5ukkBDrhQIqRDbd0&tc=fhlWhmeTV3v3lLRHv8tV4Yn3p49TcrLuAS0LSoOUU9M&cc=TMieAyqahDaYSyb1yilog54ul46wjnKlTkGncRR3SPA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl24unl-POKsciV02q7GnTSDFFx54VwMG0ujj0uRbstxBv5yqGEb8I

Successfully saved authorization token.


In [11]:
#Spectral Unmixing

# Load Landsat data
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_170052_20170108').select(bands)

# Define spectral endmembers
urban = [0.15, 0.14, 0.14, 0.15, 0.17, 0.19, 0.19]
veg = [0.098, 0.08, 0.07, 0.05, 0.34, 0.14, 0.07]
water = [0.12, 0.11, 0.14, 0.12, 0.02, 0.006, 0.005]

# Unmix the image
fractions = image.unmix([urban, veg, water])

# Reproject to Web Mercator for Folium
image = image.reproject('EPSG:4326', None, 30)
fractions = fractions.reproject('EPSG:4326', None, 30)

In [12]:
# Import pre-formated Folium code
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [14]:
# Create a folium map object.
my_map = folium.Map(location=[41.8719, 12.5674], zoom_start = 6) #ROMAITALY

# Display
my_map.add_ee_layer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.4}, 'Raw Image')
my_map.add_ee_layer(fractions, {'bands': ['band_0', 'band_1', 'band_2'], 'min': 0, 'max': 0.5}, 'Unmixed Image')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)